# 🗓️ Dollarama Shift Scheduler

This notebook uses **OR-Tools CP-SAT Solver** to generate optimal weekly schedules.

## How to Use
1. **Cell 1**: Load and view employees
2. **Cell 2**: Configure shifts (customize as needed)
3. **Cell 3**: Build and solve the constraint model
4. **Cell 4**: View the generated schedule
5. **Cell 5**: Export to JSON for the web frontend

In [ ]:
# =============================================================================
# CELL 1: IMPORTS AND LOAD EMPLOYEES
# =============================================================================

from ortools.sat.python import cp_model
import pandas as pd
import json
import os
from datetime import datetime
from IPython.display import display, HTML

EMPLOYEE_FILE = 'employees.json'

# Load employee data
if os.path.exists(EMPLOYEE_FILE):
    with open(EMPLOYEE_FILE, 'r') as f:
        employees = json.load(f)
    print(f'✅ Loaded {len(employees)} employees')
else:
    employees = []
    print('❌ No employees file found!')

# Display employee roster
print('\n' + '='*70)
print('EMPLOYEE ROSTER')
print('='*70)

roster_data = []
for e in employees:
    days_available = [d[:3] for d, t in e['availability'].items() if t]
    roster_data.append({
        'Name': e['name'],
        'Role': e['role'],
        'Target Hours': e['target_weekly_hours'],
        'Max Hours': e['max_weekly_hours'],
        'Available Days': ', '.join(days_available)
    })

roster_df = pd.DataFrame(roster_data)
display(roster_df)

In [ ]:
# =============================================================================
# CELL 2: CONFIGURE SHIFTS
# =============================================================================
# Modify this template to match your store's needs
# Format: (shift_name, start_hour, end_hour, required_role)

SHIFT_TEMPLATE = [
    ('Opener-ATL',    6, 14, 'ATL'),       # Manager opening shift
    ('Morning-FT',    6, 14, 'FullTime'),  # Full-time morning
    ('Day-FT',       10, 18, 'FullTime'),  # Full-time day shift
    ('Afternoon-PT', 13, 18, 'PartTime'),  # Part-time afternoon
    ('Closer-ATL',   14, 21, 'ATL'),       # Manager closing shift
    ('Closer-PT',    17, 21, 'PartTime'),  # Part-time closing
]

DAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Generate all shifts for the week
shifts_to_fill = []
for day in DAYS:
    for name, start, end, role in SHIFT_TEMPLATE:
        shifts_to_fill.append({
            'id': f'{day[:3]}-{name}',
            'name': name,
            'day': day,
            'start': start,
            'end': end,
            'hours': end - start,
            'role': role
        })

print(f'✅ Generated {len(shifts_to_fill)} shifts for the week')
print(f'   ({len(SHIFT_TEMPLATE)} shifts × {len(DAYS)} days)')

# Show shift breakdown by role
shift_df = pd.DataFrame(shifts_to_fill)
print('\nShifts by Role:')
print(shift_df.groupby('role')['hours'].agg(['count', 'sum']).rename(columns={'count': 'Shifts', 'sum': 'Total Hours'}))

In [ ]:
# =============================================================================
# CELL 3: BUILD AND SOLVE CONSTRAINT MODEL
# =============================================================================

print('Building constraint model...')

model = cp_model.CpModel()

# Create decision variables
# vars_dict[(e_idx, s_idx)] = 1 if employee e works shift s
vars_dict = {}
emp_shifts = {i: [] for i in range(len(employees))}  # shifts each employee can work
shift_emps = {i: [] for i in range(len(shifts_to_fill))}  # employees who can work each shift

for e_idx, emp in enumerate(employees):
    for s_idx, shift in enumerate(shifts_to_fill):
        # Check role match
        if emp['role'] != shift['role']:
            continue
        
        # Check availability
        avail = emp['availability'].get(shift['day'])
        if avail is None:
            continue
        
        # Check time window
        if shift['start'] < avail[0] or shift['end'] > avail[1]:
            continue
        
        # Create variable
        var = model.NewBoolVar(f'e{e_idx}_s{s_idx}')
        vars_dict[(e_idx, s_idx)] = var
        emp_shifts[e_idx].append(s_idx)
        shift_emps[s_idx].append(e_idx)

print(f'  ✅ Created {len(vars_dict)} feasible assignment variables')

# Check for unfillable shifts
unfillable = [shifts_to_fill[i] for i in range(len(shifts_to_fill)) if not shift_emps[i]]
if unfillable:
    print(f'\n⚠️  WARNING: {len(unfillable)} shifts cannot be filled:')
    for s in unfillable:
        print(f"    - {s['id']}: {s['day']} {s['start']:02d}:00-{s['end']:02d}:00 ({s['role']})")
    print('\n  Add employees with matching role/availability to fix this.')
else:
    print('  ✅ All shifts can be filled')

# CONSTRAINT 1: Each shift must be filled by exactly 1 person
print('\nAdding constraints...')
for s_idx in range(len(shifts_to_fill)):
    if shift_emps[s_idx]:  # Only if shift can be filled
        model.Add(sum(vars_dict[(e, s_idx)] for e in shift_emps[s_idx]) == 1)

# CONSTRAINT 2: Each employee works at most 1 shift per day
for e_idx in range(len(employees)):
    for day in DAYS:
        day_shifts = [vars_dict[(e_idx, s)] for s in emp_shifts[e_idx] 
                      if shifts_to_fill[s]['day'] == day]
        if day_shifts:
            model.Add(sum(day_shifts) <= 1)

# CONSTRAINT 3: Max weekly hours per employee
for e_idx, emp in enumerate(employees):
    hrs = [vars_dict[(e_idx, s)] * shifts_to_fill[s]['hours'] 
           for s in emp_shifts[e_idx]]
    if hrs:
        model.Add(sum(hrs) <= emp['max_weekly_hours'])

print('  ✅ Added shift coverage constraints')
print('  ✅ Added one-shift-per-day constraints')
print('  ✅ Added max weekly hours constraints')

# OBJECTIVE: Maximize hours towards each employee's target
scores = []
for e_idx, emp in enumerate(employees):
    if not emp_shifts[e_idx]:
        continue
    
    hrs = [vars_dict[(e_idx, s)] * shifts_to_fill[s]['hours'] 
           for s in emp_shifts[e_idx]]
    scheduled = model.NewIntVar(0, emp['max_weekly_hours'], f'scheduled_{e_idx}')
    model.Add(scheduled == sum(hrs))
    
    # Score is min(scheduled, target) - rewards meeting target without over-scheduling
    score = model.NewIntVar(0, emp['target_weekly_hours'], f'score_{e_idx}')
    model.AddMinEquality(score, [scheduled, emp['target_weekly_hours']])
    scores.append(score)

if scores:
    model.Maximize(sum(scores))
    print('  ✅ Objective: Maximize hours towards targets')

# SOLVE
print('\n' + '='*50)
print('SOLVING...')
print('='*50)

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 60
solver.parameters.log_search_progress = False

status = solver.Solve(model)

status_names = {
    cp_model.OPTIMAL: 'OPTIMAL ✅',
    cp_model.FEASIBLE: 'FEASIBLE ✅',
    cp_model.INFEASIBLE: 'INFEASIBLE ❌',
    cp_model.MODEL_INVALID: 'MODEL_INVALID ❌',
    cp_model.UNKNOWN: 'UNKNOWN ⚠️'
}

print(f'\nStatus: {status_names.get(status, status)}')

# Extract results
schedule_results = []
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    for (e_idx, s_idx), var in vars_dict.items():
        if solver.Value(var):
            emp = employees[e_idx]
            shift = shifts_to_fill[s_idx]
            schedule_results.append({
                'employee': emp['name'],
                'role': emp['role'],
                'day': shift['day'],
                'shift_name': shift['name'],
                'start': shift['start'],
                'end': shift['end'],
                'shift': f"{shift['start']:02d}:00-{shift['end']:02d}:00",
                'hours': shift['hours']
            })
    
    print(f'\n🎉 SUCCESS! Generated schedule with {len(schedule_results)} shift assignments')
    print(f'   Solve time: {solver.WallTime():.2f} seconds')
else:
    print('\n❌ Could not generate schedule. Check employee availability.')

In [ ]:
# =============================================================================
# CELL 4: DISPLAY SCHEDULE
# =============================================================================

if not schedule_results:
    print('❌ No schedule generated. Run Cell 3 first.')
else:
    df = pd.DataFrame(schedule_results)
    
    # Weekly Grid View
    print('='*80)
    print('WEEKLY SCHEDULE GRID')
    print('='*80)
    
    grid = df.pivot_table(
        index='employee', 
        columns='day', 
        values='shift', 
        aggfunc='first'
    )
    grid = grid[DAYS].fillna('---')
    display(grid)
    
    # Hours Summary
    print('\n' + '='*80)
    print('HOURS SUMMARY')
    print('='*80)
    
    hrs_summary = df.groupby('employee')['hours'].sum().reset_index()
    hrs_summary.columns = ['Employee', 'Scheduled']
    
    emp_df = pd.DataFrame(employees)
    summary = hrs_summary.merge(
        emp_df[['name', 'role', 'target_weekly_hours', 'max_weekly_hours']], 
        left_on='Employee', 
        right_on='name'
    )
    summary = summary[['Employee', 'role', 'Scheduled', 'target_weekly_hours', 'max_weekly_hours']]
    summary.columns = ['Employee', 'Role', 'Scheduled', 'Target', 'Max']
    summary['Diff'] = summary['Scheduled'] - summary['Target']
    
    # Color code the diff column
    def highlight_diff(val):
        if val >= 0:
            return 'background-color: #c6efce; color: #006100'
        else:
            return 'background-color: #ffc7ce; color: #9c0006'
    
    styled_summary = summary.style.applymap(highlight_diff, subset=['Diff'])
    display(styled_summary)
    
    # Totals
    total_shifts = len(schedule_results)
    total_hours = summary['Scheduled'].sum()
    print(f'\n📊 TOTALS: {total_shifts} shifts | {total_hours} hours scheduled')

In [ ]:
# =============================================================================
# CELL 5: EXPORT FOR WEB FRONTEND
# =============================================================================

if not schedule_results:
    print('❌ No schedule to export. Run Cell 3 first.')
else:
    # Build export data structure
    df = pd.DataFrame(schedule_results)
    
    # Calculate hours summary
    hrs_by_emp = df.groupby('employee')['hours'].sum().to_dict()
    
    # Build employee summary
    emp_summary = []
    for emp in employees:
        scheduled = hrs_by_emp.get(emp['name'], 0)
        emp_summary.append({
            'name': emp['name'],
            'role': emp['role'],
            'scheduled_hours': int(scheduled),
            'target_hours': emp['target_weekly_hours'],
            'max_hours': emp['max_weekly_hours']
        })
    
    export_data = {
        'generated': datetime.now().isoformat(),
        'week_of': datetime.now().strftime('%Y-%m-%d'),
        'stats': {
            'total_shifts': len(schedule_results),
            'total_hours': int(df['hours'].sum()),
            'employees_scheduled': len(hrs_by_emp)
        },
        'employees': emp_summary,
        'schedule': schedule_results,
        'days': DAYS,
        'shifts': [s['name'] for s in SHIFT_TEMPLATE]
    }
    
    # Save to docs folder for GitHub Pages
    output_path = 'docs/schedule_output.json'
    os.makedirs('docs', exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print(f'✅ Exported schedule to: {output_path}')
    print(f'\n📄 Export Summary:')
    print(f'   - Generated: {export_data["generated"]}')
    print(f'   - Total Shifts: {export_data["stats"]["total_shifts"]}')
    print(f'   - Total Hours: {export_data["stats"]["total_hours"]}')
    print(f'   - Employees: {export_data["stats"]["employees_scheduled"]}')
    
    # Also save CSV versions for Excel users
    prefix = f"schedule_{datetime.now().strftime('%Y%m%d_%H%M')}"
    
    grid = df.pivot_table(index='employee', columns='day', values='shift', aggfunc='first')
    grid = grid[[d for d in DAYS if d in grid.columns]].fillna('---')
    grid.to_csv(f'{prefix}_grid.csv')
    
    print(f'\n📁 Also saved: {prefix}_grid.csv')